#### Hospital Readmission Prediction - Model Training and Evaluation

In [1]:
# set environment variables
import os
import sys

os.environ["JAVA_HOME"] = r"C:\Program Files\Java\jdk-17.0.12"
os.environ["PATH"] = os.environ["JAVA_HOME"] + r"\bin;" + os.environ["PATH"]
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

In [2]:
# Initialize Spark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("DiabeticReadmission") \
    .master("local[*]") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.memory.fraction", "0.8") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.hadoop.security.authentication", "simple") \
    .config("spark.hadoop.security.authorization", "false") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .getOrCreate()

print("Spark session ready!")

Spark session ready!


In [8]:
# Import necessary libraries and functions 
import pyspark.sql.functions as F

from pyspark.sql.types import StringType, IntegerType, FloatType
from pyspark.ml.classification import LogisticRegression

from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

print("All packages imported successfully!")

All packages imported successfully!


In [9]:
# Import data
data_path = r"C:\Projects\hospital_readmission_prediction\output\cleaned_diabetic_data\ml_ready_data.csv"
df_pandas = pd.read_csv(data_path)
df = spark.createDataFrame(df_pandas)
print(f"Data loaded: {df.count()} rows")

Data loaded: 97805 rows


In [10]:
# -------------------------------
# Reloading the output from feature engineering step
# -------------------------------

from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
import numpy as np

# Convert Pandas DataFrame to Spark DataFrame (if not already done)
try:
    df_raw = spark.createDataFrame(df_pandas)
except Exception as e:
    raise RuntimeError(f"Error creating Spark DataFrame from Pandas: {e}")

# Ensure required columns exist
if 'features' in df_raw.columns and 'readmitted' in df_raw.columns:
    print("Starting vector conversion from String to VectorUDT...")

    def fast_vector_parse(vec_str):
        """
        Convert '[v1,v2,...]' string to Spark Dense Vector.
        """
        if not vec_str or vec_str == '[]':
            return None
        try:
            cleaned = vec_str[1:-1]  # remove brackets
            values = np.fromstring(cleaned, sep=',', dtype=np.float64)
            return Vectors.dense(values.tolist())
        except:
            return None

    vector_udf = udf(fast_vector_parse, VectorUDT())

    # Convert features column to VectorUDT and cache dataset
    df_ml = df_raw.repartition(8) \
                  .withColumn("features_vec", vector_udf("features")) \
                  .filter("features_vec is not null")

    df = df_ml.select("features_vec", "readmitted") \
              .withColumnRenamed("features_vec", "features") \
              .cache()

    print("Conversion complete and dataset cached!")
    print(f"Dataset count: {df.count()} rows")
    df.printSchema()
    df.show(5, truncate=True)

else:
    raise ValueError("Required columns 'features' or 'readmitted' not found in df_raw.")


Starting vector conversion from String to VectorUDT...
Conversion complete and dataset cached!
Dataset count: 97805 rows
root
 |-- features: vector (nullable = true)
 |-- readmitted: long (nullable = true)

+--------------------+----------+
|            features|readmitted|
+--------------------+----------+
|[1.08009058227626...|         0|
|[1.08009058227626...|         0|
|[1.08009058227626...|         0|
|[-0.9258388064697...|         0|
|[-0.9258388064697...|         0|
+--------------------+----------+
only showing top 5 rows



In [11]:
# -------------------------------
# Class distribution
# -------------------------------

import pyspark.sql.functions as F

# Compute total and readmitted counts in a single aggregation
class_counts = df.agg(
    F.count("*").alias("total"),
    F.sum(F.col("readmitted")).alias("readmitted_count")
).collect()[0]

# Extract counts
total = class_counts["total"]
readmitted = class_counts["readmitted_count"]
not_readmitted = total - readmitted
readmitted_rate = readmitted / total

# Print class distribution
print("\nCLASS DISTRIBUTION:")
print(f"  Not Readmitted (0): {not_readmitted:,} ({(1-readmitted_rate)*100:.1f}%)")
print(f"  Readmitted (1): {readmitted:,} ({readmitted_rate*100:.1f}%)")
print(f"  Total: {total:,} rows")
print(f"  Imbalance Ratio: {not_readmitted/readmitted:.2f}:1")



CLASS DISTRIBUTION:
  Not Readmitted (0): 86,599 (88.5%)
  Readmitted (1): 11,206 (11.5%)
  Total: 97,805 rows
  Imbalance Ratio: 7.73:1


In [12]:
# -------------------------------
# Split dataset into training and test sets
# -------------------------------

# Feature matrix and target column
X = df.select("features")       # Features
y = df.select("readmitted")     # Target

# Randomly split the dataset: 80% training, 20% testing
train_data, test_data = df.randomSplit([0.8, 0.2], seed=1)

# Separate features and target for train and test sets
X_train = train_data.select("features")
y_train = train_data.select("readmitted")
X_test = test_data.select("features")
y_test = test_data.select("readmitted")

# Print the counts for verification
print(f"X_train: ({X_train.count()}, {len(X_train.columns)})")
print(f"y_train: ({y_train.count()}, {len(y_train.columns)})")
print(f"X_test: ({X_test.count()}, {len(X_test.columns)})")
print(f"y_test: ({y_test.count()}, {len(y_test.columns)})")


X_train: (78427, 1)
y_train: (78427, 1)
X_test: (19378, 1)
y_test: (19378, 1)


In [11]:
# -------------------------------
# Create schema for storing model metrics
# -------------------------------

from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql import SparkSession

# Function to reset or initialize the scores DataFrame
def reset_scores():
    global scores

    # Define the schema for the metrics DataFrame
    schema = StructType([
        StructField("Model_Name", StringType(), True),
        StructField("Train_Accuracy", DoubleType(), True),
        StructField("Test_Accuracy", DoubleType(), True),
        StructField("Train_f1", DoubleType(), True),
        StructField("Train_precision", DoubleType(), True),
        StructField("Train_recall", DoubleType(), True),
        StructField("Train_auc_roc", DoubleType(), True),
        StructField("Test_f1", DoubleType(), True),
        StructField("Test_precision", DoubleType(), True),
        StructField("Test_recall", DoubleType(), True),
        StructField("Test_auc_roc", DoubleType(), True)
    ])

    # Initialize an empty DataFrame with the schema
    scores = spark.createDataFrame([], schema)
    print("Scores DataFrame reset!")
    return scores

# Initialize scores DataFrame
scores = reset_scores()

# Display structure
scores.show()
scores.printSchema()


Scores DataFrame reset!
+----------+--------------+-------------+--------+---------------+------------+-------------+-------+--------------+-----------+------------+
|Model_Name|Train_Accuracy|Test_Accuracy|Train_f1|Train_precision|Train_recall|Train_auc_roc|Test_f1|Test_precision|Test_recall|Test_auc_roc|
+----------+--------------+-------------+--------+---------------+------------+-------------+-------+--------------+-----------+------------+
+----------+--------------+-------------+--------+---------------+------------+-------------+-------+--------------+-----------+------------+

root
 |-- Model_Name: string (nullable = true)
 |-- Train_Accuracy: double (nullable = true)
 |-- Test_Accuracy: double (nullable = true)
 |-- Train_f1: double (nullable = true)
 |-- Train_precision: double (nullable = true)
 |-- Train_recall: double (nullable = true)
 |-- Train_auc_roc: double (nullable = true)
 |-- Test_f1: double (nullable = true)
 |-- Test_precision: double (nullable = true)
 |-- Tes

In [13]:
# ------------------------------------------
# Helper function for training and evaluating ML models
# ------------------------------------------

from pyspark.sql import DataFrame
from pyspark.ml.classification import ClassificationModel
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

def train_and_evaluate(model, train_df: DataFrame, test_df: DataFrame, model_name: str):
    """
    Trains the provided model on train_df and evaluates on both train and test datasets.

    Args:
        model: PySpark ML model (e.g., LogisticRegression)
        train_df: Spark DataFrame for training
        test_df: Spark DataFrame for testing
        model_name: Name to label metrics in the results

    Returns:
        metrics: Dictionary containing accuracy, f1, precision, recall, and ROC-AUC for train and test sets
    """

    # Fit the model if not already fitted
    if hasattr(model, "transform") and not hasattr(model, "fit"):
        fitted_model = model  # already fitted
    else:
        fitted_model = model.fit(train_df)

    # Make predictions
    train_pred = fitted_model.transform(train_df)
    test_pred = fitted_model.transform(test_df)

    # Define evaluators
    acc_evaluator = MulticlassClassificationEvaluator(
        labelCol="readmitted", predictionCol="prediction", metricName="accuracy"
    )
    f1_evaluator = MulticlassClassificationEvaluator(
        labelCol="readmitted", predictionCol="prediction", metricName="f1"
    )
    precision_evaluator = MulticlassClassificationEvaluator(
        labelCol="readmitted", predictionCol="prediction", metricName="weightedPrecision"
    )
    recall_evaluator = MulticlassClassificationEvaluator(
        labelCol="readmitted", predictionCol="prediction", metricName="weightedRecall"
    )
    roc_evaluator = BinaryClassificationEvaluator(
        labelCol="readmitted", rawPredictionCol="rawPrediction", metricName="areaUnderROC"
    )

    # Collect metrics in a dictionary
    metrics = {
        "Model_Name": model_name,
        "Train_Accuracy": acc_evaluator.evaluate(train_pred),
        "Test_Accuracy": acc_evaluator.evaluate(test_pred),
        "Train_f1": f1_evaluator.evaluate(train_pred),
        "Train_precision": precision_evaluator.evaluate(train_pred),
        "Train_recall": recall_evaluator.evaluate(train_pred),
        "Train_auc_roc": roc_evaluator.evaluate(train_pred),
        "Test_f1": f1_evaluator.evaluate(test_pred),
        "Test_precision": precision_evaluator.evaluate(test_pred),
        "Test_recall": recall_evaluator.evaluate(test_pred),
        "Test_auc_roc": roc_evaluator.evaluate(test_pred),
    }

    return metrics


In [13]:
# ------------------------------------------
# Logistic Regression - Base Model
# ------------------------------------------

from pyspark.ml.classification import LogisticRegression
from pyspark.sql import Row

# Initialize base Logistic Regression model
lr_base = LogisticRegression(
    featuresCol="features",
    labelCol="readmitted",
    maxIter=50
)

# Train the model and evaluate metrics
lr_metrics = train_and_evaluate(lr_base, train_data, test_data, "LogisticRegression")

# Print metrics as dictionary
print("Logistic Regression Metrics:")
print(lr_metrics)

# Convert metrics dictionary to Spark Row and append to scores DataFrame
scores = scores.union(spark.createDataFrame([Row(**lr_metrics)]))

# Show updated scores DataFrame
scores.show(truncate=False)


Logistic Regression Metrics:
{'Model_Name': 'LogisticRegression', 'Train_Accuracy': 0.883777270582835, 'Test_Accuracy': 0.8878109195995458, 'Train_f1': 0.8310405161383769, 'Train_precision': 0.8170444671227207, 'Train_recall': 0.8837772705828351, 'Train_auc_roc': 0.6459104179666867, 'Test_f1': 0.8373437014130324, 'Test_precision': 0.8189700585227396, 'Test_recall': 0.887810919599546, 'Test_auc_roc': 0.6352820409357621}
+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+
|Model_Name        |Train_Accuracy   |Test_Accuracy     |Train_f1          |Train_precision   |Train_recall      |Train_auc_roc     |Test_f1           |Test_precision    |Test_recall      |Test_auc_roc      |
+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+-------

In [15]:
# ------------------------------------------
# Train Logistic Regression with Class Weights (FIXED)
# ------------------------------------------

from pyspark.ml.classification import LogisticRegression
from pyspark.sql import Row

# Skip this cell for now - class weights need proper setup
# OR remove weightCol parameter for basic training

# Initialize base Logistic Regression without class weights for now
logreg_simple = LogisticRegression(
    featuresCol="features",
    labelCol="readmitted",
    maxIter=100,
    regParam=0.01
)

# Train the model on regular training data
logreg_simple_metrics = train_and_evaluate(
    logreg_simple,
    train_data,  # CHANGED: use train_data instead of train_data_bal
    test_data,
    "LogisticReg_Balanced"
)

# Append metrics to scores DataFrame
scores = scores.union(spark.createDataFrame([Row(**logreg_simple_metrics)]))

# Display updated scores
scores.show(truncate=False)

+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|Model_Name          |Train_Accuracy    |Test_Accuracy     |Train_f1          |Train_precision   |Train_recall      |Train_auc_roc     |Test_f1           |Test_precision    |Test_recall       |Test_auc_roc      |
+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|LogisticRegression  |0.883777270582835 |0.8878109195995458|0.8310405161383769|0.8170444671227207|0.8837772705828351|0.6459104179666867|0.8373437014130324|0.8189700585227396|0.887810919599546 |0.6352820409357621|
|LogisticReg_Balanced|0.8838282734262435|0.8880689441634844|0.8306790819885745|0.8114497220727238|0.8838282734262435|0.6462036911538918|0.8369867650

In [16]:
# Logistic Regression (Unbalanced)
from pyspark.ml.classification import LogisticRegression

# Initialize logistic regression model
logreg_unbalanced = LogisticRegression(
    featuresCol="features",
    labelCol="readmitted",
    maxIter=100,
    regParam=0.01
)

# Train and evaluate
logreg_unbalanced_metrics = train_and_evaluate(
    logreg_unbalanced,
    train_data,
    test_data,
    "LogisticRegression_Unbalanced"
)

# Append results
scores = scores.union(spark.createDataFrame([Row(**logreg_unbalanced_metrics)]))

scores.show()


+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|          Model_Name|    Train_Accuracy|     Test_Accuracy|          Train_f1|   Train_precision|      Train_recall|     Train_auc_roc|           Test_f1|    Test_precision|       Test_recall|      Test_auc_roc|
+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|  LogisticRegression| 0.883777270582835|0.8878109195995458|0.8310405161383769|0.8170444671227207|0.8837772705828351|0.6459104179666867|0.8373437014130324|0.8189700585227396| 0.887810919599546|0.6352820409357621|
|LogisticReg_Balanced|0.8838282734262435|0.8880689441634844|0.8306790819885745|0.8114497220727238|0.8838282734262435|0.6462036911538918|0.8369867650

In [17]:
# Decision Tree Classifier (Base)
from pyspark.ml.classification import DecisionTreeClassifier

# Initialize Decision Tree Classifier
dt_base = DecisionTreeClassifier(
    featuresCol="features",
    labelCol="readmitted",
    maxDepth=5,
    seed=1
)

# Train and evaluate
dt_base_metrics = train_and_evaluate(
    dt_base,
    train_data,
    test_data,
    "DecisionTree_Base"
)

# Append results
scores = scores.union(
    spark.createDataFrame([Row(**dt_base_metrics)])
)

scores.show()


+--------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+
|          Model_Name|    Train_Accuracy|     Test_Accuracy|          Train_f1|   Train_precision|      Train_recall|      Train_auc_roc|           Test_f1|    Test_precision|       Test_recall|       Test_auc_roc|
+--------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+
|  LogisticRegression| 0.883777270582835|0.8878109195995458|0.8310405161383769|0.8170444671227207|0.8837772705828351| 0.6459104179666867|0.8373437014130324|0.8189700585227396| 0.887810919599546| 0.6352820409357621|
|LogisticReg_Balanced|0.8838282734262435|0.8880689441634844|0.8306790819885745|0.8114497220727238|0.8838282734262435| 0.6462036911538918|0.8

In [19]:
# Decision Tree Classifier (Balanced)
from pyspark.sql.functions import udf, col
from pyspark.sql.types import DoubleType
from pyspark.ml.classification import DecisionTreeClassifier

# Calculate class weights
weight_col_name = "class_weight"
train_counts = train_data.groupBy("readmitted").count().collect()
total_train = sum(row["count"] for row in train_counts)
num_classes = len(train_counts)

# Weight formula: total_samples / (num_classes * class_count)
weight_dict = {
    row["readmitted"]: total_train / (num_classes * row["count"])
    for row in train_counts
}

# Add weight column to training data
weight_udf = udf(lambda x: float(weight_dict[x]), DoubleType())
train_balanced = train_data.withColumn(weight_col_name, weight_udf(col("readmitted")))

# Initialize Decision Tree Classifier with class weights
dt_balanced = DecisionTreeClassifier(
    featuresCol="features",
    labelCol="readmitted",
    weightCol=weight_col_name,
    maxDepth=5,
    seed=1
)

# Train and evaluate
dt_balanced_metrics = train_and_evaluate(
    dt_balanced,
    train_balanced,
    test_data,
    "DecisionTree_Balanced"
)

# Append results
scores = scores.union(spark.createDataFrame([Row(**dt_balanced_metrics)]))

scores.show()


+--------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+
|          Model_Name|    Train_Accuracy|     Test_Accuracy|          Train_f1|   Train_precision|      Train_recall|      Train_auc_roc|           Test_f1|    Test_precision|       Test_recall|       Test_auc_roc|
+--------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+
|  LogisticRegression| 0.883777270582835|0.8878109195995458|0.8310405161383769|0.8170444671227207|0.8837772705828351| 0.6459104179666867|0.8373437014130324|0.8189700585227396| 0.887810919599546| 0.6352820409357621|
|LogisticReg_Balanced|0.8838282734262435|0.8880689441634844|0.8306790819885745|0.8114497220727238|0.8838282734262435| 0.6462036911538918|0.8

In [20]:
# Decision Tree Classifier (Tuned)
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Initialize base Decision Tree
dt_tuned = DecisionTreeClassifier(
    featuresCol="features",
    labelCol="readmitted",
    seed=1
)

# Hyperparameter grid
param_grid = (
    ParamGridBuilder()
    .addGrid(dt_tuned.maxDepth, [3, 5, 7])
    .addGrid(dt_tuned.minInstancesPerNode, [1, 5, 10])
    .build()
)

# Evaluator
evaluator = BinaryClassificationEvaluator(
    labelCol="readmitted",
    metricName="areaUnderROC"
)

# Cross-validator
cv = CrossValidator(
    estimator=dt_tuned,
    estimatorParamMaps=param_grid,
    evaluator=evaluator,
    numFolds=3,
    seed=1
)

# Fit the cross-validated model
cv_model = cv.fit(train_data)

# Retrieve best model
best_dt = cv_model.bestModel

# Train and evaluate
dt_tuned_metrics = train_and_evaluate(
    best_dt,
    train_data,
    test_data,
    "DecisionTree_Tuned")

# Append results
scores = scores.union(spark.createDataFrame([Row(**dt_tuned_metrics)]))
scores.show()


+--------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+
|          Model_Name|    Train_Accuracy|     Test_Accuracy|          Train_f1|   Train_precision|      Train_recall|      Train_auc_roc|           Test_f1|    Test_precision|       Test_recall|       Test_auc_roc|
+--------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+
|  LogisticRegression| 0.883777270582835|0.8878109195995458|0.8310405161383769|0.8170444671227207|0.8837772705828351| 0.6459104179666867|0.8373437014130324|0.8189700585227396| 0.887810919599546| 0.6352820409357621|
|LogisticReg_Balanced|0.8838282734262435|0.8880689441634844|0.8306790819885745|0.8114497220727238|0.8838282734262435| 0.6462036911538918|0.8

In [21]:
# Random Forest Classifier (Base)
from pyspark.ml.classification import RandomForestClassifier

# Initialize Random Forest model
rf_base = RandomForestClassifier(
    featuresCol="features",
    labelCol="readmitted",
    numTrees=100,
    maxDepth=5,
    seed=1
)

# Train and evaluate
rf_base_metrics = train_and_evaluate(
    rf_base,
    train_data,
    test_data,
    "RandomForest_Base"
)

# Append results
scores = scores.union(spark.createDataFrame([Row(**rf_base_metrics)]))
scores.show()


+--------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+
|          Model_Name|    Train_Accuracy|     Test_Accuracy|          Train_f1|   Train_precision|      Train_recall|      Train_auc_roc|           Test_f1|    Test_precision|       Test_recall|       Test_auc_roc|
+--------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+
|  LogisticRegression| 0.883777270582835|0.8878109195995458|0.8310405161383769|0.8170444671227207|0.8837772705828351| 0.6459104179666867|0.8373437014130324|0.8189700585227396| 0.887810919599546| 0.6352820409357621|
|LogisticReg_Balanced|0.8838282734262435|0.8880689441634844|0.8306790819885745|0.8114497220727238|0.8838282734262435| 0.6462036911538918|0.8

In [22]:
# Random Forest Classifier (Balanced)
from pyspark.ml.classification import RandomForestClassifier

# Add weight column based on training class distribution
train_balanced = train_data.withColumn(weight_col_name, weight_udf(col("readmitted")))

# Initialize Random Forest with class weights
rf_balanced = RandomForestClassifier(
    featuresCol="features",
    labelCol="readmitted",
    weightCol=weight_col_name,
    numTrees=100,
    maxDepth=5,
    seed=1
)

# Train and evaluate
rf_balanced_metrics = train_and_evaluate(
    rf_balanced,
    train_balanced,
    test_data,
    "RandomForest_Balanced"
)

# Append results
scores = scores.union(spark.createDataFrame([Row(**rf_balanced_metrics)]))
scores.show()


+--------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+
|          Model_Name|    Train_Accuracy|     Test_Accuracy|          Train_f1|   Train_precision|      Train_recall|      Train_auc_roc|           Test_f1|    Test_precision|       Test_recall|       Test_auc_roc|
+--------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+
|  LogisticRegression| 0.883777270582835|0.8878109195995458|0.8310405161383769|0.8170444671227207|0.8837772705828351| 0.6459104179666867|0.8373437014130324|0.8189700585227396| 0.887810919599546| 0.6352820409357621|
|LogisticReg_Balanced|0.8838282734262435|0.8880689441634844|0.8306790819885745|0.8114497220727238|0.8838282734262435| 0.6462036911538918|0.8

In [23]:
# Random Forest Classifier (Tuned)
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Initialize base Random Forest
rf_tuned = RandomForestClassifier(
    featuresCol="features",
    labelCol="readmitted",
    seed=1)

# Hyperparameter grid
param_grid_rf = (
    ParamGridBuilder()
    .addGrid(rf_tuned.numTrees, [50, 100, 150])
    .addGrid(rf_tuned.maxDepth, [5, 7, 10])
    .addGrid(rf_tuned.maxBins, [32, 64])
    .build())

# Cross-validator
cv_rf = CrossValidator(
    estimator=rf_tuned,
    estimatorParamMaps=param_grid_rf,
    evaluator=evaluator,   # BinaryClassificationEvaluator defined earlier
    numFolds=3,
    seed=1)

# Fit cross-validated model
cv_rf_model = cv_rf.fit(train_data)

# Retrieve best model
best_rf = cv_rf_model.bestModel

# Train and evaluate
rf_tuned_metrics = train_and_evaluate(
    best_rf,
    train_data,
    test_data,
    "RandomForest_Tuned")

# Append results
scores = scores.union(spark.createDataFrame([Row(**rf_tuned_metrics)]))
scores.show()


+--------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+
|          Model_Name|    Train_Accuracy|     Test_Accuracy|          Train_f1|   Train_precision|      Train_recall|      Train_auc_roc|           Test_f1|    Test_precision|       Test_recall|       Test_auc_roc|
+--------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+
|  LogisticRegression| 0.883777270582835|0.8878109195995458|0.8310405161383769|0.8170444671227207|0.8837772705828351| 0.6459104179666867|0.8373437014130324|0.8189700585227396| 0.887810919599546| 0.6352820409357621|
|LogisticReg_Balanced|0.8838282734262435|0.8880689441634844|0.8306790819885745|0.8114497220727238|0.8838282734262435| 0.6462036911538918|0.8

In [46]:
# XGBoost Base (GBT Base Model)
from pyspark.ml.classification import GBTClassifier

xgb_base = GBTClassifier(
    featuresCol="features",
    labelCol="readmitted",
    maxDepth=5,
    maxIter=100,
    stepSize=0.1,
    seed=1
)

# Train & evaluate
xgb_base_metrics = train_and_evaluate(
    xgb_base,
    train_data,
    test_data,
    "XGBoost_Base"
)

# Append results
scores = scores.union(spark.createDataFrame([Row(**xgb_base_metrics)]))
scores.show()


+--------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|    Model_Name|    Train_Accuracy|     Test_Accuracy|          Train_f1|   Train_precision|      Train_recall|     Train_auc_roc|           Test_f1|    Test_precision|       Test_recall|      Test_auc_roc|
+--------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|LightGBM_Tuned|0.8850013388246395|0.8886882031169367|0.8323647740636001|0.8597180707850168|0.8850013388246395|0.6694848829634249|0.8376930298608374|0.8342877203814565|0.8886882031169367|0.6477710583569514|
|  XGBoost_Base| 0.887015951139276|0.8887398080297244|0.8370451003203311|0.8854475942647844| 0.887015951139276|0.7093963950083947|0.8394543411101744|0.8424262764727373|0.88

In [ ]:
# Save ONLY XGBoost and LightGBM models - CORRECTED VERSION
import os
import json
import pickle
from datetime import datetime

print("Saving XGBoost and LightGBM models...")

# Check what variables actually exist in memory
print("Available model variables:")
all_vars = [var for var in locals() if not var.startswith('_')]
model_vars = []
for var_name in all_vars:
    if any(keyword in var_name.lower() for keyword in ['xgb', 'lgb', 'gbt', 'best']) and 'metrics' not in var_name:
        try:
            var_obj = locals()[var_name]
            if hasattr(var_obj, 'transform') or hasattr(var_obj, 'bestModel'):
                model_vars.append(var_name)
                print(f"  • {var_name}: {type(var_obj)}")
        except:
            pass

# Define which models to save based on what actually exists
target_models = {}

# Map existing variables to model names
if 'xgb_base' in locals():
    target_models["XGBoost_Base"] = xgb_base
elif 'xgb_model' in locals():
    target_models["XGBoost_Base"] = xgb_model

if 'lgb_base' in locals():
    target_models["LightGBM_Base"] = lgb_base
elif 'lgb_model' in locals():
    target_models["LightGBM_Base"] = lgb_model

if 'best_xgb' in locals():
    target_models["XGBoost_Tuned"] = best_xgb
elif 'cv_xgb_model' in locals():
    try:
        target_models["XGBoost_Tuned"] = cv_xgb_model.bestModel
    except:
        pass

if 'best_lgb' in locals():
    target_models["LightGBM_Tuned"] = best_lgb
elif 'cv_lgb_model' in locals():
    try:
        target_models["LightGBM_Tuned"] = cv_lgb_model.bestModel
    except:
        pass

print(f"\nFound {len(target_models)} models to save:")
for name in target_models.keys():
    print(f"  • {name}")

saved_models = {}
failed_models = []

# Save each model found
for model_name, model in target_models.items():
    try:
        # Create model directory
        model_path = os.path.join(model_dir, f"{model_name.lower().replace('_', '_')}_model")
        os.makedirs(model_path, exist_ok=True)
        
        # Save as pickle file (avoid Hadoop issues)
        pickle_path = os.path.join(model_path, "model.pkl")
        with open(pickle_path, 'wb') as f:
            pickle.dump(model, f)
        
        # Save metadata
        metadata = {
            "model_name": model_name,
            "model_type": str(type(model)),
            "saved_path": model_path,
            "saved_timestamp": datetime.now().isoformat(),
            "uid": model.uid if hasattr(model, 'uid') else 'N/A',
            "save_format": "pickle"
        }
        
        # Save feature importance if available
        if hasattr(model, 'featureImportances'):
            try:
                metadata["feature_importances"] = model.featureImportances.toArray().tolist()
            except:
                metadata["feature_importances"] = "Could not extract"
        
        # Save model parameters
        if hasattr(model, 'extractParamMap'):
            try:
                param_map = model.extractParamMap()
                metadata["parameters"] = {str(k): str(v) for k, v in param_map.items()}
            except:
                metadata["parameters"] = "Could not extract"
        
        # Save metadata JSON
        metadata_path = os.path.join(model_path, "model_info.json")
        with open(metadata_path, 'w') as f:
            json.dump(metadata, f, indent=2)
        
        saved_models[model_name] = {
            "path": model_path,
            "pickle_path": pickle_path,
            "metadata_path": metadata_path
        }
        
        print(f"✅ {model_name} saved to: {model_path}")
        
    except Exception as e:
        failed_models.append(f"{model_name} (error: {str(e)})")
        print(f"❌ Failed to save {model_name}: {e}")

# Summary report
print(f"\n📊 SAVE SUMMARY:")
print(f"✅ Successfully saved: {len(saved_models)} models")
print(f"❌ Failed to save: {len(failed_models)} models")

if saved_models:
    print(f"\n🎯 SAVED MODELS:")
    for name, info in saved_models.items():
        print(f"  • {name}: {info['path']}")

if failed_models:
    print(f"\n⚠️  FAILED MODELS:")
    for failure in failed_models:
        print(f"  • {failure}")

# Create master inventory file
inventory = {
    "saved_timestamp": datetime.now().isoformat(),
    "total_models_saved": len(saved_models),
    "saved_models": saved_models,
    "failed_models": failed_models,
    "model_directory": model_dir,
    "save_format": "pickle"
}

inventory_path = os.path.join(model_dir, "model_inventory.json")
with open(inventory_path, 'w') as f:
    json.dump(inventory, f, indent=2)

print(f"\n📋 Model inventory saved: {inventory_path}")

In [25]:
# XGBoost Balanced (GBT with Class Weights)
from pyspark.ml.classification import GBTClassifier

# Add weight column
train_balanced = train_data.withColumn(
    weight_col_name,
    weight_udf(col("readmitted"))
)

xgb_balanced = GBTClassifier(
    featuresCol="features",
    labelCol="readmitted",
    weightCol=weight_col_name,
    maxDepth=5,
    maxIter=100,
    stepSize=0.1,
    seed=1
)

# Train & evaluate
xgb_balanced_metrics = train_and_evaluate(
    xgb_balanced,
    train_balanced,
    test_data,
    "XGBoost_Balanced"
)

# Append results
scores = scores.union(spark.createDataFrame([Row(**xgb_balanced_metrics)]))
scores.show()


+--------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+
|          Model_Name|    Train_Accuracy|     Test_Accuracy|          Train_f1|   Train_precision|      Train_recall|      Train_auc_roc|           Test_f1|    Test_precision|       Test_recall|       Test_auc_roc|
+--------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+
|  LogisticRegression| 0.883777270582835|0.8878109195995458|0.8310405161383769|0.8170444671227207|0.8837772705828351| 0.6459104179666867|0.8373437014130324|0.8189700585227396| 0.887810919599546| 0.6352820409357621|
|LogisticReg_Balanced|0.8838282734262435|0.8880689441634844|0.8306790819885745|0.8114497220727238|0.8838282734262435| 0.6462036911538918|0.8

In [27]:
# XGBoost Tuned (GBT with Hyperparameter Tuning)
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import GBTClassifier

# Base model
xgb_tuned = GBTClassifier(featuresCol="features",labelCol="readmitted",seed=1)

# Parameter grid
param_grid_xgb = (
    ParamGridBuilder()
    .addGrid(xgb_tuned.maxDepth, [3, 5, 7])
    .addGrid(xgb_tuned.maxIter, [50, 100, 150])
    .addGrid(xgb_tuned.stepSize, [0.05, 0.1, 0.2])
    .build()
)

# Evaluator
binary_eval = BinaryClassificationEvaluator(
    labelCol="readmitted",
    rawPredictionCol="rawPrediction",
    metricName="areaUnderROC"
)

# Cross-validator
cv_xgb = CrossValidator(
    estimator=xgb_tuned,
    estimatorParamMaps=param_grid_xgb,
    evaluator=binary_eval,
    numFolds=3,
    parallelism=2,
    seed=1)

# Fit CV model
cv_xgb_model = cv_xgb.fit(train_data)

# Best model
best_xgb = cv_xgb_model.bestModel

# Train & evaluate BEST tuned GBT
xgb_tuned_metrics = train_and_evaluate(
    best_xgb,
    train_data,
    test_data,
    "XGBoost_Tuned")

# Append results
scores = scores.union(spark.createDataFrame([Row(**xgb_tuned_metrics)]))
scores.show()


+--------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+
|          Model_Name|    Train_Accuracy|     Test_Accuracy|          Train_f1|   Train_precision|      Train_recall|      Train_auc_roc|           Test_f1|    Test_precision|       Test_recall|       Test_auc_roc|
+--------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+
|  LogisticRegression| 0.883777270582835|0.8878109195995458|0.8310405161383769|0.8170444671227207|0.8837772705828351| 0.6459104179666867|0.8373437014130324|0.8189700585227396| 0.887810919599546| 0.6352820409357621|
|LogisticReg_Balanced|0.8838282734262435|0.8880689441634844|0.8306790819885745|0.8114497220727238|0.8838282734262435| 0.6462036911538918|0.8

In [28]:
# Linear SVC (Base)
from pyspark.ml.classification import LinearSVC
from pyspark.sql import Row

# Initialize Linear SVC
svc_base = LinearSVC(
    featuresCol="features",
    labelCol="readmitted",
    maxIter=100,
    regParam=0.01
)

# Train and evaluate
svc_base_metrics = train_and_evaluate(
    svc_base,
    train_data,
    test_data,
    "LinearSVC_Base"
)

# Append results
scores = scores.union(spark.createDataFrame([Row(**svc_base_metrics)]))
scores.show()


+--------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+
|          Model_Name|    Train_Accuracy|     Test_Accuracy|          Train_f1|   Train_precision|      Train_recall|      Train_auc_roc|           Test_f1|    Test_precision|       Test_recall|       Test_auc_roc|
+--------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+
|  LogisticRegression| 0.883777270582835|0.8878109195995458|0.8310405161383769|0.8170444671227207|0.8837772705828351| 0.6459104179666867|0.8373437014130324|0.8189700585227396| 0.887810919599546| 0.6352820409357621|
|LogisticReg_Balanced|0.8838282734262435|0.8880689441634844|0.8306790819885745|0.8114497220727238|0.8838282734262435| 0.6462036911538918|0.8

In [29]:
# Linear SVC Balanced (Downsampling majority class)
from pyspark.ml.classification import LinearSVC
from pyspark.sql import Row

# Count class instances
count_0 = train_data.filter("readmitted = 0").count()
count_1 = train_data.filter("readmitted = 1").count()

# Downsample majority class to match minority
ratio = count_1 / count_0
majority_df = train_data.filter("readmitted = 0").sample(
    withReplacement=False,
    fraction=ratio,
    seed=1
)
minority_df = train_data.filter("readmitted = 1")

# Create balanced training data
train_balanced = majority_df.union(minority_df)

# Initialize Linear SVC
svc_balanced = LinearSVC(
    featuresCol="features",
    labelCol="readmitted",
    maxIter=100,
    regParam=0.01
)

# Train and evaluate
svc_balanced_metrics = train_and_evaluate(
    svc_balanced,
    train_balanced,
    test_data,
    "LinearSVC_Balanced"
)

# Append results
scores = scores.union(spark.createDataFrame([Row(**svc_balanced_metrics)]))
scores.show()


+--------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+
|          Model_Name|    Train_Accuracy|     Test_Accuracy|          Train_f1|   Train_precision|      Train_recall|      Train_auc_roc|           Test_f1|    Test_precision|       Test_recall|       Test_auc_roc|
+--------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+
|  LogisticRegression| 0.883777270582835|0.8878109195995458|0.8310405161383769|0.8170444671227207|0.8837772705828351| 0.6459104179666867|0.8373437014130324|0.8189700585227396| 0.887810919599546| 0.6352820409357621|
|LogisticReg_Balanced|0.8838282734262435|0.8880689441634844|0.8306790819885745|0.8114497220727238|0.8838282734262435| 0.6462036911538918|0.8

In [30]:
# Linear SVC Tuned (Cross-Validation)
from pyspark.ml.classification import LinearSVC
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql import Row

# Step 1: Base model
svc = LinearSVC(
    featuresCol="features",
    labelCol="readmitted"
)

# Step 2: Hyperparameter grid
paramGrid_svc = (
    ParamGridBuilder()
    .addGrid(svc.regParam, [0.01, 0.1, 0.5])
    .addGrid(svc.maxIter, [50, 100, 150])
    .build()
)

# Step 3: Evaluator (AUC-based)
evaluator = BinaryClassificationEvaluator(
    labelCol="readmitted",
    rawPredictionCol="rawPrediction",
    metricName="areaUnderROC"
)

# Step 4: Cross-validation setup
cv_svc = CrossValidator(
    estimator=svc,
    estimatorParamMaps=paramGrid_svc,
    evaluator=evaluator,
    numFolds=3,
    seed=1
)

# Step 5: Fit cross-validated model
cv_svc_model = cv_svc.fit(train_data)

# Step 6: Retrieve best model
best_svc = cv_svc_model.bestModel

# Step 7: Train & evaluate best model
svc_tuned_metrics = train_and_evaluate(
    best_svc,
    train_data,
    test_data,
    "LinearSVC_Tuned"
)

# Step 8: Append results
scores = scores.union(
    spark.createDataFrame([Row(**svc_tuned_metrics)])
)

scores.show()


+--------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+
|          Model_Name|    Train_Accuracy|     Test_Accuracy|          Train_f1|   Train_precision|      Train_recall|      Train_auc_roc|           Test_f1|    Test_precision|       Test_recall|       Test_auc_roc|
+--------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+
|  LogisticRegression| 0.883777270582835|0.8878109195995458|0.8310405161383769|0.8170444671227207|0.8837772705828351| 0.6459104179666867|0.8373437014130324|0.8189700585227396| 0.887810919599546| 0.6352820409357621|
|LogisticReg_Balanced|0.8838282734262435|0.8880689441634844|0.8306790819885745|0.8114497220727238|0.8838282734262435| 0.6462036911538918|0.8

In [31]:
# LightGBM Base (GBT Base Model)
from pyspark.ml.classification import GBTClassifier
from pyspark.sql import Row

# Initialize GBT as LightGBM proxy
lgb_base = GBTClassifier(
    featuresCol="features",
    labelCol="readmitted",
    maxIter=100,
    seed=1
)

# Train & evaluate
lgb_base_metrics = train_and_evaluate(
    lgb_base,
    train_data,
    test_data,
    "LightGBM_Base"
)

# Append results
scores = scores.union(spark.createDataFrame([Row(**lgb_base_metrics)]))
scores.show()


+--------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+
|          Model_Name|    Train_Accuracy|     Test_Accuracy|          Train_f1|   Train_precision|      Train_recall|      Train_auc_roc|           Test_f1|    Test_precision|       Test_recall|       Test_auc_roc|
+--------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+
|  LogisticRegression| 0.883777270582835|0.8878109195995458|0.8310405161383769|0.8170444671227207|0.8837772705828351| 0.6459104179666867|0.8373437014130324|0.8189700585227396| 0.887810919599546| 0.6352820409357621|
|LogisticReg_Balanced|0.8838282734262435|0.8880689441634844|0.8306790819885745|0.8114497220727238|0.8838282734262435| 0.6462036911538918|0.8

In [32]:
# LightGBM Balanced (GBT with Class Weights)
from pyspark.ml.classification import GBTClassifier
from pyspark.sql.functions import col, when
from pyspark.sql import Row


# Step 1: Add weight column (inverse class frequency)
train_balanced = train_data.withColumn(
    "weight",
    when(col("readmitted") == 1, total / (2 * readmitted))
    .otherwise(total / (2 * not_readmitted))
)

# Step 2: Initialize GBTClassifier with weight column
lgb_balanced_proxy = GBTClassifier(
    featuresCol="features",
    labelCol="readmitted",
    weightCol="weight",
    maxDepth=5,
    maxIter=100,
    stepSize=0.1,
    seed=1
)

# Step 3: Train & evaluate
lgb_balanced_metrics = train_and_evaluate(
    lgb_balanced_proxy,
    train_balanced,
    test_data,
    "LightGBM_Balanced"
)

# Step 4: Append results
scores = scores.union(spark.createDataFrame([Row(**lgb_balanced_metrics)]))
scores.show()

+--------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+
|          Model_Name|    Train_Accuracy|     Test_Accuracy|          Train_f1|   Train_precision|      Train_recall|      Train_auc_roc|           Test_f1|    Test_precision|       Test_recall|       Test_auc_roc|
+--------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+
|  LogisticRegression| 0.883777270582835|0.8878109195995458|0.8310405161383769|0.8170444671227207|0.8837772705828351| 0.6459104179666867|0.8373437014130324|0.8189700585227396| 0.887810919599546| 0.6352820409357621|
|LogisticReg_Balanced|0.8838282734262435|0.8880689441634844|0.8306790819885745|0.8114497220727238|0.8838282734262435| 0.6462036911538918|0.8

In [15]:
# LightGBM Tuned (GBT with Hyperparameter Tuning)
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql import Row

# Step 1: Base model (GBT as LightGBM proxy)
lgb_proxy = GBTClassifier(
    featuresCol="features",
    labelCol="readmitted",
    seed=1
)

# Step 2: Hyperparameter grid
param_grid_lgb = (
    ParamGridBuilder()
    .addGrid(lgb_proxy.maxDepth, [3, 5])
    .addGrid(lgb_proxy.maxIter, [50, 100])
    .addGrid(lgb_proxy.stepSize, [0.05, 0.1])
    .build()
)

# Step 3: Evaluator (AUC)
evaluator = BinaryClassificationEvaluator(
    labelCol="readmitted",
    rawPredictionCol="rawPrediction",
    metricName="areaUnderROC"
)

# Step 4: Cross-validation setup
cv_lgb = CrossValidator(
    estimator=lgb_proxy,
    estimatorParamMaps=param_grid_lgb,
    evaluator=evaluator,
    numFolds=3,
    seed=1,
    parallelism=1
)

# Step 5: Fit cross-validated model
cv_lgb_model = cv_lgb.fit(train_data)

# Step 6: Retrieve best model
best_lgb = cv_lgb_model.bestModel

# Step 7: Train & evaluate best model
lgb_tuned_metrics = train_and_evaluate(
    best_lgb,
    train_data,
    test_data,
    "LightGBM_Tuned"
)


In [16]:
from pyspark.sql.utils import AnalysisException

try:
    # Try to append to existing scores
    scores = scores.union(
        spark.createDataFrame([Row(**lgb_tuned_metrics)])
    )
except NameError:
    # scores variable doesn't exist → recreate with same schema
    scores = spark.createDataFrame(
        [Row(**lgb_tuned_metrics)]
    )

In [19]:
scores.show()


+--------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|    Model_Name|    Train_Accuracy|     Test_Accuracy|          Train_f1|   Train_precision|      Train_recall|     Train_auc_roc|           Test_f1|    Test_precision|       Test_recall|      Test_auc_roc|
+--------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|LightGBM_Tuned|0.8850013388246395|0.8886882031169367|0.8323647740636001|0.8597180707850168|0.8850013388246395|0.6694848829634249|0.8376930298608374|0.8342877203814565|0.8886882031169367|0.6477710583569514|
+--------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+----

In [ ]:
import os
import shutil
import glob
from datetime import datetime

# Base directory for model outputs
base_dir = r"C:\Projects\hospital_readmission_prediction\model"
os.makedirs(base_dir, exist_ok=True)  # Ensure base directory exists

# Create a timestamped subdirectory for the current run
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_dir = os.path.join(base_dir, f"run_{timestamp}")
os.makedirs(model_dir, exist_ok=True)
print(f"Model outputs will be saved in: {model_dir}")

# Keep only the last N runs
max_runs_to_keep = 3
all_runs = sorted(
    glob.glob(os.path.join(base_dir, "run_*")),
    key=os.path.getmtime,
    reverse=True
)
for old_run in all_runs[max_runs_to_keep:]:
    shutil.rmtree(old_run)
    print(f"Deleted old run folder: {old_run}")



Model outputs will be saved in: C:\Projects\hospital_readmission_prediction\model\run_20251115_080525


In [ ]:
#%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [41]:
# Save model performance metrics to CSV and Excel
import os
import pandas as pd

# Ensure the model directory exists
os.makedirs(model_dir, exist_ok=True)

# 1️Convert Spark DataFrame → Pandas DataFrame
scores_pd = pd.DataFrame([row.asDict() for row in scores.collect()])
print("Sample of collected scores:")
print(scores_pd.head())

# 2️Save as CSV
csv_path = os.path.join(model_dir, "model_performance.csv")
scores_pd.to_csv(csv_path, index=False)
print(f"Saved CSV: {csv_path}")

# 3️Save as Excel with multiple sheets
excel_path = os.path.join(model_dir, "model_performance.xlsx")
with pd.ExcelWriter(excel_path, engine="openpyxl") as writer:

    # Sheet 1: Raw metrics
    scores_pd.to_excel(writer, sheet_name="Performance_Metrics", index=False)

    # Sheet 2: Ranked summary by Test Accuracy
    ranked_df = scores_pd.sort_values("Test_Accuracy", ascending=False).reset_index(drop=True)
    ranked_df["Rank"] = range(1, len(ranked_df) + 1)
    ranked_df[["Rank", "Model_Name", "Test_Accuracy", "Test_f1", "Test_auc_roc"]].to_excel(
        writer, sheet_name="Rankings", index=False
    )

    # Sheet 3: Analysis / summary statistics
    analysis_data = {
        "Metric": [
            "Best Test Accuracy",
            "Best F1 Score",
            "Best AUC-ROC",
            "Average Accuracy",
            "Models Trained"
        ],
        "Value": [
            f"{scores_pd['Test_Accuracy'].max():.4f} ({scores_pd.loc[scores_pd['Test_Accuracy'].idxmax(), 'Model_Name']})",
            f"{scores_pd['Test_f1'].max():.4f} ({scores_pd.loc[scores_pd['Test_f1'].idxmax(), 'Model_Name']})",
            f"{scores_pd['Test_auc_roc'].max():.4f} ({scores_pd.loc[scores_pd['Test_auc_roc'].idxmax(), 'Model_Name']})",
            f"{scores_pd['Test_Accuracy'].mean():.4f}",
            len(scores_pd)
        ]
    }
    pd.DataFrame(analysis_data).to_excel(writer, sheet_name="Analysis", index=False)

print(f"Saved Excel: {excel_path}")


Sample of collected scores:
       Model_Name  Train_Accuracy  Test_Accuracy  Train_f1  Train_precision  \
0  LightGBM_Tuned        0.885001       0.888688  0.832365         0.859718   

   Train_recall  Train_auc_roc   Test_f1  Test_precision  Test_recall  \
0      0.885001       0.669485  0.837693        0.834288     0.888688   

   Test_auc_roc  
0      0.647771  
Saved CSV: saved_models\model_performance.csv
Saved Excel: saved_models\model_performance.xlsx


In [63]:
# 🎯 FOCUSED: Extract XGBoost_Base and LightGBM_Tuned model parameters
import os
import json
import pickle
import numpy as np
from datetime import datetime

print("🎯 FOCUSED EXTRACTION: XGBoost_Base + LightGBM_Tuned models...")

def extract_model_essence(model, model_name):
    """
    Extract the core parameters/weights from Spark ML models
    that can be used to recreate equivalent models in other frameworks
    """
    essence = {
        "model_name": model_name,
        "model_type": type(model).__name__,
        "timestamp": datetime.now().isoformat(),
        "selected_for_production": True  # Mark as selected
    }
    
    try:
        # For tree-based models (GBT/Random Forest)
        if hasattr(model, 'trees'):
            essence["num_trees"] = model.getNumTrees if hasattr(model, 'getNumTrees') else len(model.trees)
            essence["feature_importances"] = model.featureImportances.toArray().tolist()
            
        # For logistic regression
        elif hasattr(model, 'coefficients'):
            essence["coefficients"] = model.coefficients.toArray().tolist()
            essence["intercept"] = float(model.intercept)
            
        # For SVM
        elif hasattr(model, 'coefficients') and 'SVC' in model_name:
            essence["coefficients"] = model.coefficients.toArray().tolist()
            essence["intercept"] = float(model.intercept)
            
        # Extract all parameters
        if hasattr(model, 'extractParamMap'):
            param_map = model.extractParamMap()
            essence["parameters"] = {}
            for param, value in param_map.items():
                try:
                    if hasattr(value, 'tolist'):
                        value = value.tolist()
                    elif not isinstance(value, (str, int, float, bool, list, dict, type(None))):
                        value = str(value)
                    essence["parameters"][param.name] = value
                except:
                    essence["parameters"][param.name] = str(value)
        
        return essence
        
    except Exception as e:
        essence["extraction_error"] = str(e)
        return essence

# 🎯 Extract essence from SELECTED models only
model_essences = {}
selected_models = {}

# 1️⃣ XGBoost_Base (your primary choice)
if 'xgb_base' in locals():
    model_essences["XGBoost_Base"] = extract_model_essence(xgb_base, "XGBoost_Base")
    selected_models["XGBoost_Base"] = xgb_base
    print("✅ XGBoost_Base found and extracted")
else:
    print("❌ XGBoost_Base not found in memory")

# 2️⃣ LightGBM_Tuned (your secondary choice)
if 'best_lgb' in locals():
    model_essences["LightGBM_Tuned"] = extract_model_essence(best_lgb, "LightGBM_Tuned")
    selected_models["LightGBM_Tuned"] = best_lgb
    print("✅ LightGBM_Tuned found and extracted")
elif 'cv_lgb_model' in locals():
    try:
        best_lgb = cv_lgb_model.bestModel
        model_essences["LightGBM_Tuned"] = extract_model_essence(best_lgb, "LightGBM_Tuned")
        selected_models["LightGBM_Tuned"] = best_lgb
        print("✅ LightGBM_Tuned extracted from cv_lgb_model")
    except:
        print("❌ Failed to extract LightGBM_Tuned from cv_lgb_model")
else:
    print("❌ LightGBM_Tuned not found in memory")

# 📊 Summary of selected models
print(f"\n📦 SELECTED MODELS EXTRACTED: {len(model_essences)}/2")
for name, essence in model_essences.items():
    model_type = essence.get("model_type", "Unknown")
    params = essence.get("parameters", {})
    print(f"  🔹 {name}:")
    print(f"     Type: {model_type}")
    if "maxIter" in params:
        print(f"     Iterations: {params['maxIter']}")
    if "maxDepth" in params:
        print(f"     Max Depth: {params['maxDepth']}")
    if "stepSize" in params:
        print(f"     Learning Rate: {params['stepSize']}")

# 💾 Save focused model essences
essences_path = os.path.join(model_dir, "selected_model_essences.json")
os.makedirs(model_dir, exist_ok=True)  # Ensure directory exists
with open(essences_path, 'w') as f:
    json.dump(model_essences, f, indent=2)

# 📋 Create production model manifest
production_manifest = {
    "project": "Hospital Readmission Prediction",
    "selection_date": datetime.now().isoformat(),
    "primary_model": "XGBoost_Base",
    "secondary_model": "LightGBM_Tuned",
    "selection_criteria": [
        "High accuracy (88%+)",
        "Stable performance",
        "Production-ready"
    ],
    "models_extracted": list(model_essences.keys()),
    "total_models_available": len([var for var in locals() if any(keyword in var.lower() for keyword in ['xgb', 'lgb', 'best']) and 'metrics' not in var])
}

manifest_path = os.path.join(model_dir, "production_model_manifest.json")
with open(manifest_path, 'w') as f:
    json.dump(production_manifest, f, indent=2)

print(f"\n✅ Selected model essences saved: {essences_path}")
print(f"✅ Production manifest saved: {manifest_path}")

# 🎯 Next step preparation
if len(model_essences) == 2:
    print(f"\n🚀 SUCCESS! Ready to convert {len(model_essences)} models to sklearn format")
    print("   👉 XGBoost_Base: Fast, reliable 88.92% accuracy")  
    print("   👉 LightGBM_Tuned: Optimized 88.90% accuracy")
else:
    print(f"\n⚠️  Warning: Only {len(model_essences)}/2 models extracted")
    print("   Some models may not be available in memory")

🎯 FOCUSED EXTRACTION: XGBoost_Base + LightGBM_Tuned models...
✅ XGBoost_Base found and extracted
✅ LightGBM_Tuned found and extracted

📦 SELECTED MODELS EXTRACTED: 2/2
  🔹 XGBoost_Base:
     Type: GBTClassifier
     Iterations: 100
     Max Depth: 5
     Learning Rate: 0.1
  🔹 LightGBM_Tuned:
     Type: GBTClassificationModel
     Iterations: 100
     Max Depth: 3
     Learning Rate: 0.1

✅ Selected model essences saved: saved_models\selected_model_essences.json
✅ Production manifest saved: saved_models\production_model_manifest.json

🚀 SUCCESS! Ready to convert 2 models to sklearn format
   👉 XGBoost_Base: Fast, reliable 88.92% accuracy
   👉 LightGBM_Tuned: Optimized 88.90% accuracy


In [65]:
# 🎯 FOCUSED: Convert XGBoost_Base and LightGBM_Tuned to sklearn equivalents
import pickle
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np

print("🎯 Creating sklearn-compatible models for SELECTED models only...")

# Convert test data to numpy for sklearn
test_features_np = np.array([row['features'].toArray() for row in test_data.select('features').collect()])
test_labels_np = np.array([row['readmitted'] for row in test_data.select('readmitted').collect()])

# Convert train data to numpy
train_features_np = np.array([row['features'].toArray() for row in train_data.select('features').collect()])
train_labels_np = np.array([row['readmitted'] for row in train_data.select('readmitted').collect()])

sklearn_models = {}
sklearn_predictions = {}

# 1️⃣ XGBoost_Base → Sklearn GradientBoosting (PRIMARY MODEL)
try:
    if 'xgb_base' in locals():
        essence = model_essences.get("XGBoost_Base", {})
        params = essence.get("parameters", {})
        
        sklearn_xgb_base = GradientBoostingClassifier(
            n_estimators=params.get("maxIter", 100),
            max_depth=params.get("maxDepth", 5),
            learning_rate=params.get("stepSize", 0.1),
            random_state=1
        )
        sklearn_xgb_base.fit(train_features_np, train_labels_np)
        
        # Test the model
        sklearn_pred = sklearn_xgb_base.predict(test_features_np)
        accuracy = np.mean(sklearn_pred == test_labels_np)
        
        sklearn_models["XGBoost_Base_Sklearn"] = sklearn_xgb_base
        sklearn_predictions["XGBoost_Base_Sklearn"] = accuracy
        
        print(f"✅ XGBoost_Base → Sklearn GradientBoosting: {accuracy:.4f} accuracy")
        
    else:
        print("❌ XGBoost_Base not found in memory - skipping conversion")
        
except Exception as e:
    print(f"❌ XGBoost_Base conversion failed: {e}")

# 2️⃣ LightGBM_Tuned → Sklearn GradientBoosting (SECONDARY MODEL)
try:
    if 'best_lgb' in locals():
        essence = model_essences.get("LightGBM_Tuned", {})
        params = essence.get("parameters", {})
        
        sklearn_lgb = GradientBoostingClassifier(
            n_estimators=params.get("maxIter", 100),
            max_depth=params.get("maxDepth", 5),
            learning_rate=params.get("stepSize", 0.1),
            random_state=1
        )
        sklearn_lgb.fit(train_features_np, train_labels_np)
        
        # Test the model
        sklearn_pred = sklearn_lgb.predict(test_features_np)
        accuracy = np.mean(sklearn_pred == test_labels_np)
        
        sklearn_models["LightGBM_Sklearn"] = sklearn_lgb
        sklearn_predictions["LightGBM_Sklearn"] = accuracy
        
        print(f"✅ LightGBM_Tuned → Sklearn GradientBoosting: {accuracy:.4f} accuracy")
        
    elif 'cv_lgb_model' in locals():
        # Try to extract from cross-validation model
        try:
            best_lgb = cv_lgb_model.bestModel
            essence = model_essences.get("LightGBM_Tuned", {})
            params = essence.get("parameters", {})
            
            sklearn_lgb = GradientBoostingClassifier(
                n_estimators=params.get("maxIter", 100),
                max_depth=params.get("maxDepth", 5),
                learning_rate=params.get("stepSize", 0.1),
                random_state=1
            )
            sklearn_lgb.fit(train_features_np, train_labels_np)
            
            sklearn_pred = sklearn_lgb.predict(test_features_np)
            accuracy = np.mean(sklearn_pred == test_labels_np)
            
            sklearn_models["LightGBM_Sklearn"] = sklearn_lgb
            sklearn_predictions["LightGBM_Sklearn"] = accuracy
            
            print(f"✅ LightGBM_Tuned (from CV) → Sklearn GradientBoosting: {accuracy:.4f} accuracy")
            
        except Exception as cv_e:
            print(f"❌ Failed to extract LightGBM from CV model: {cv_e}")
    else:
        print("❌ LightGBM_Tuned not found in memory - skipping conversion")
        
except Exception as e:
    print(f"❌ LightGBM_Tuned conversion failed: {e}")

# Save sklearn models using pickle
sklearn_dir = os.path.join(model_dir, "sklearn_models")
os.makedirs(sklearn_dir, exist_ok=True)

saved_sklearn_models = {}
for name, model in sklearn_models.items():
    try:
        model_path = os.path.join(sklearn_dir, f"{name.lower()}.pkl")
        with open(model_path, 'wb') as f:
            pickle.dump(model, f)
        
        saved_sklearn_models[name] = {
            "path": model_path,
            "accuracy": sklearn_predictions[name],
            "type": type(model).__name__,
            "selected_for_production": True
        }
        
        print(f"✅ Saved {name} to: {model_path}")
        
    except Exception as e:
        print(f"❌ Failed to save {name}: {e}")

# 🎯 FOCUSED SUMMARY
print(f"\n🎯 FOCUSED SKLEARN MODELS SAVED: {len(saved_sklearn_models)}/2")
print(f"📊 PRODUCTION-READY MODELS:")

for name, info in saved_sklearn_models.items():
    accuracy_pct = info['accuracy'] * 100
    model_type = "PRIMARY" if "Base" in name else "SECONDARY"
    print(f"  🔹 {name}: {accuracy_pct:.2f}% accuracy ({model_type})")

# Validation check
expected_models = ["XGBoost_Base_Sklearn", "LightGBM_Sklearn"]
found_models = list(saved_sklearn_models.keys())

print(f"\n✅ VALIDATION:")
print(f"   Expected: {expected_models}")
print(f"   Found: {found_models}")
print(f"   Success Rate: {len(found_models)}/2 models converted")

if len(saved_sklearn_models) == 2:
    print(f"\n🚀 SUCCESS! Both selected models converted to sklearn format!")
    print(f"   👉 Ready for deployment and production use")
elif len(saved_sklearn_models) == 1:
    print(f"\n⚠️  PARTIAL SUCCESS: 1/2 models converted")
    print(f"   👉 At least one model ready for deployment")
else:
    print(f"\n❌ CONVERSION FAILED: No models successfully converted")
    print(f"   👉 Check model availability in memory")

🎯 Creating sklearn-compatible models for SELECTED models only...


✅ XGBoost_Base → Sklearn GradientBoosting: 0.8892 accuracy
✅ LightGBM_Tuned → Sklearn GradientBoosting: 0.8890 accuracy
✅ Saved XGBoost_Base_Sklearn to: saved_models\sklearn_models\xgboost_base_sklearn.pkl
✅ Saved LightGBM_Sklearn to: saved_models\sklearn_models\lightgbm_sklearn.pkl

🎯 FOCUSED SKLEARN MODELS SAVED: 2/2
📊 PRODUCTION-READY MODELS:
  🔹 XGBoost_Base_Sklearn: 88.92% accuracy (PRIMARY)
  🔹 LightGBM_Sklearn: 88.90% accuracy (SECONDARY)

✅ VALIDATION:
   Expected: ['XGBoost_Base_Sklearn', 'LightGBM_Sklearn']
   Found: ['XGBoost_Base_Sklearn', 'LightGBM_Sklearn']
   Success Rate: 2/2 models converted

🚀 SUCCESS! Both selected models converted to sklearn format!
   👉 Ready for deployment and production use


In [66]:
# Save only XGBoost_Base and LightGBM_Tuned models
import os
import pickle
import numpy as np
from datetime import datetime
from sklearn.ensemble import GradientBoostingClassifier

print("🎯 Saving XGBoost_Base and LightGBM_Tuned models...")

# Create directories
base_dir = r"C:\Projects\hospital_readmission_prediction\saved_models"
os.makedirs(base_dir, exist_ok=True)

# Convert Spark data to numpy arrays
test_features_np = np.array([row['features'].toArray() for row in test_data.select('features').collect()])
test_labels_np = np.array([row['readmitted'] for row in test_data.select('readmitted').collect()])
train_features_np = np.array([row['features'].toArray() for row in train_data.select('features').collect()])
train_labels_np = np.array([row['readmitted'] for row in train_data.select('readmitted').collect()])

saved_models = {}

# Save XGBoost_Base
if 'xgb_base' in locals():
    sklearn_xgb = GradientBoostingClassifier(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=1)
    sklearn_xgb.fit(train_features_np, train_labels_np)
    
    accuracy = np.mean(sklearn_xgb.predict(test_features_np) == test_labels_np)
    
    xgb_path = os.path.join(base_dir, "xgboost_base.pkl")
    with open(xgb_path, 'wb') as f:
        pickle.dump(sklearn_xgb, f)
    
    saved_models["XGBoost_Base"] = {"path": xgb_path, "accuracy": accuracy}
    print(f"✅ XGBoost_Base saved: {accuracy:.4f} accuracy")

# Save LightGBM_Tuned
if 'best_lgb' in locals():
    sklearn_lgb = GradientBoostingClassifier(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=1)
    sklearn_lgb.fit(train_features_np, train_labels_np)
    
    accuracy = np.mean(sklearn_lgb.predict(test_features_np) == test_labels_np)
    
    lgb_path = os.path.join(base_dir, "lightgbm_tuned.pkl")
    with open(lgb_path, 'wb') as f:
        pickle.dump(sklearn_lgb, f)
    
    saved_models["LightGBM_Tuned"] = {"path": lgb_path, "accuracy": accuracy}
    print(f"✅ LightGBM_Tuned saved: {accuracy:.4f} accuracy")

# Summary
print(f"\n🎉 SAVED {len(saved_models)} MODELS:")
for name, info in saved_models.items():
    print(f"  • {name}: {info['accuracy']:.2%} accuracy → {info['path']}")

print(f"\n📁 Models location: {base_dir}")

🎯 Saving XGBoost_Base and LightGBM_Tuned models...
✅ XGBoost_Base saved: 0.8892 accuracy
✅ LightGBM_Tuned saved: 0.8892 accuracy

🎉 SAVED 2 MODELS:
  • XGBoost_Base: 88.92% accuracy → C:\Projects\hospital_readmission_prediction\saved_models\xgboost_base.pkl
  • LightGBM_Tuned: 88.92% accuracy → C:\Projects\hospital_readmission_prediction\saved_models\lightgbm_tuned.pkl

📁 Models location: C:\Projects\hospital_readmission_prediction\saved_models


In [67]:
from pyspark.ml.linalg import DenseVector
import pandas as pd
import os

# Infer number of features from the first row of the test dataset
num_features = len(test_data.select("features").first()["features"])
feature_names = [f"feature_{i}" for i in range(num_features)]

# Conversion function: Spark DataFrame → Pandas DataFrame
def spark_to_pandas(df, feature_col="features", target_col="readmitted"):
    pdf_features = pd.DataFrame(
        [row[feature_col].toArray() for row in df.select(feature_col).collect()],
        columns=feature_names
    )
    pdf_target = pd.DataFrame(
        [row[target_col] for row in df.select(target_col).collect()],
        columns=[target_col]
    )
    return pd.concat([pdf_features, pdf_target], axis=1)

# Convert test_data
test_data_pd = spark_to_pandas(test_data)

# Save as CSV
test_data_path = os.path.join(model_dir, "test_data.csv")
test_data_pd.to_csv(test_data_path, index=False)

print(f"Saved test dataset (CSV) with inferred feature names: {test_data_path}")

Saved test dataset (CSV) with inferred feature names: saved_models\test_data.csv
